In [1]:
pwd

'C:\\Users\\Prashant\\Downloads\\AV\\SBSA 2023\\Upto ML Basics\\Probability'

In [1]:
import pandas as pd
df = pd.read_pickle("churn_prediction_v3.pkl")
df.head()

,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,current_balance,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,2401,66,Male,0,self_employed,187,2,755,1458.71,1458.71,1458.71,1449.07,0.20,0.20,0.20,0.20,1458.71,1458.71,0.0
1,2648,35,Male,0,self_employed,100000,2,3214,5390.37,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0.0
2,2494,31,Male,0,salaried,146,2,41,3913.16,5815.29,4910.17,2815.94,0.61,0.61,6046.73,259.23,5006.28,5070.14,0.0
3,2629,90,Male,1,self_employed,1020,2,582,2291.91,2291.91,2084.54,1006.54,0.47,0.47,0.47,2143.33,2291.91,1669.79,1.0
4,1879,42,Male,2,self_employed,1494,3,388,927.72,1401.72,1643.31,1871.12,0.33,714.61,588.62,1538.06,1157.15,1677.16,1.0


In [29]:
df.dtypes

vintage                              int64
age                                  int64
gender                            category
dependents                           int64
occupation                        category
city                              category
customer_nw_category              category
branch_code                       category
current_balance                    float64
previous_month_end_balance         float64
average_monthly_balance_prevQ      float64
average_monthly_balance_prevQ2     float64
current_month_credit               float64
previous_month_credit              float64
current_month_debit                float64
previous_month_debit               float64
current_month_balance              float64
previous_month_balance             float64
churn                             category
dtype: object

# Example of HT on Proportions: churn rate is higher for "self-employed" occupation

In [21]:
import numpy as np
import pandas as pd
from scipy import stats

# p1 = churn rate for "self-employed"
# p2 = churn rate for other occupations

# Step 1: Formulate the null and alternative hypotheses
# Null Hypothesis (H0): p1 ≤ p2 (churn rates are the same or less for "self-employed" and other occupations)
# Alternative Hypothesis (HA): p1 > p2 (churn rate is higher for "self-employed" occupation)

# Step 2: Collect and prepare the data
occupation_churn = pd.crosstab(df['occupation'], df['churn'])
occupation_churn

churn,0.0,1.0
occupation,,
company,22,1
retired,967,178
salaried,3081,664
self_employed,7905,1955
student,967,189


In [22]:
# Step 3: Choose a significance level (α)
alpha = 0.05

In [23]:
# Step 4: Compute the test statistic
self_employed_churn = occupation_churn.loc['self_employed', 1]
other_occupations_churn = occupation_churn.loc[['salaried', 'student', 'retired'], 1].sum()

self_employed_count = occupation_churn.loc['self_employed'].sum()
other_occupations_count = occupation_churn.loc[['salaried', 'student', 'retired']].sum().sum()

p1 = self_employed_churn / self_employed_count
p2 = other_occupations_churn / other_occupations_count
p1, p2

(0.19827586206896552, 0.1705259675818723)

In [24]:
standard_error = np.sqrt((p1 * (1 - p1) / self_employed_count) + (p2 * (1 - p2) / other_occupations_count))

test_statistic = (p1 - p2) / standard_error
test_statistic

4.414372046085935

In [25]:
# Step 5: Determine the critical value or p-value
critical_value = norm.isf(alpha)  # For Right-tailed test
critical_value

1.6448536269514729

In [26]:
p_value = norm.sf(test_statistic)  # For Right-tailed test
p_value

5.065180563732681e-06

In [27]:
# Step 6: Make a decision
if test_statistic > critical_value or p_value < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")

# Step 7: Draw a conclusion
if test_statistic > critical_value or p_value < alpha:
    print("There is evidence to suggest that the churn rate is higher for the 'self-employed' occupation")
else:
    print("There is insufficient evidence to suggest a difference in churn rates between the 'self-employed' occupation and other occupations")

Reject the null hypothesis
There is evidence to suggest that the churn rate is higher for the 'self-employed' occupation


# Hypothesis: Customers with a higher current month debit have a higher churn rate.

In [32]:
import numpy as np
import pandas as pd
from scipy import stats

# Hypothesis: Customers with a higher current month debit have a higher churn rate.


# Step 1: Formulate the null and alternative hypotheses
# Null Hypothesis (H0): μ ≤ 5000 (current month debit has no effect on churn)
# Alternative Hypothesis (HA): μ > 5000 (customers with higher current month debit have a higher churn rate)

# Step 2: Collect and prepare the data
current_month_debit_churn = df[df['churn'] == 1]['current_month_debit']

# Step 3: Choose a significance level (α)
alpha = 0.05

# Step 4: Compute the test statistic
sample_mean = current_month_debit_churn.mean()
population_mean = 5000
sample_size = current_month_debit_churn.size
sample_std = current_month_debit_churn.std()

standard_error = sample_std / np.sqrt(sample_size)
test_statistic = (sample_mean - population_mean) / standard_error

# Step 5: Determine the critical value or p-value
critical_value = stats.norm.ppf(1 - alpha)  # For one-tailed test
p_value = 1 - stats.norm.cdf(test_statistic)  # For one-tailed test

# Step 6: Make a decision
if test_statistic > critical_value or p_value < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")

# Step 7: Draw a conclusion
if test_statistic > critical_value or p_value < alpha:
    print("There is evidence to suggest that customers with higher current month debit have a higher churn rate")
else:
    print("There is insufficient evidence to suggest a difference in churn rates based on current month debit")


Reject the null hypothesis
There is evidence to suggest that customers with higher current month debit have a higher churn rate


# Example of HT on Proportions: Churn rate is higher for "customer_nw_category" category 2

In [36]:
customer_nw_category_2_churn

/usr/local/lib/python3.8/dist-packages/IPython/lib/pretty.py:778: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  output = repr(obj)


0        0.0
1        0.0
2        0.0
3        1.0
5        0.0
        ... 
15922    0.0
15924    0.0
15926    1.0
15927    0.0
15928    0.0
Name: churn, Length: 8163, dtype: category
Categories (2, float64): [0.0, 1.0]

In [38]:
import numpy as np
import pandas as pd
from scipy import stats

# Step 1: Formulate the null and alternative hypotheses
# Null Hypothesis (H0): p1 ≤ p2 (churn rates are the same for "customer_nw_category" category 2 and other categories)
# Alternative Hypothesis (HA): p1 > p2 (churn rate is higher for "customer_nw_category" category 2)

# Step 2: Collect and prepare the data
customer_nw_category_2_churn = df[df['customer_nw_category'].astype("int32") == 2]['churn'].astype("int32")

# Step 3: Choose a significance level (α)
alpha = 0.05

# Step 4: Compute the test statistic
p1 = customer_nw_category_2_churn.mean()
p2 = df['churn'].astype("int32").mean()
n1 = customer_nw_category_2_churn.size
n2 = df['churn'].size

test_statistic = (p1 - p2) / np.sqrt((p1 * (1 - p1) / n1) + (p2 * (1 - p2) / n2))

# Step 5: Determine the critical value or p-value
critical_value = stats.norm.ppf(1 - alpha)  # For one-tailed test
p_value = 1 - stats.norm.cdf(test_statistic)  # For one-tailed test

# Step 6: Make a decision
if test_statistic > critical_value or p_value < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")

# Step 7: Draw a conclusion
if test_statistic > critical_value or p_value < alpha:
    print("There is evidence to suggest that customers in 'customer_nw_category' category 2 have a higher churn rate")
else:
    print("There is insufficient evidence to suggest a difference in churn rates based on 'customer_nw_category' category")


Fail to reject the null hypothesis
There is insufficient evidence to suggest a difference in churn rates based on 'customer_nw_category' category


# Example of HT on Proportions: Gender has no effect on churn

In [17]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Step 1: Formulate the null and alternative hypotheses
# Null Hypothesis (H0): p1 = p2 (gender has no effect on churn)
# Alternative Hypothesis (HA): p1 ≠ p2 (gender has an effect on churn)

In [5]:
pd.crosstab(df['gender'], df['churn'])

churn,0.0,1.0
gender,,
Female,5208,1126
Male,7734,1861


In [6]:
# Step 2: Collect and prepare the data
male_churn = df[df['gender'] == 'Male']['churn'].astype("int32")
female_churn = df[df['gender'] == 'Female']['churn'].astype("int32")

In [8]:
female_churn

5        0
14       1
15       0
17       0
18       0
        ..
15921    1
15923    0
15925    1
15927    0
15928    0
Name: churn, Length: 6334, dtype: int32

In [9]:
# Step 3: Choose a significance level (α)
alpha = 0.05

In [10]:
# Step 4: Compute the test statistic
p1 = male_churn.mean()
p2 = female_churn.mean()

p1, p2

(0.19395518499218342, 0.1777707609725292)

In [11]:
# P(Churn==1 | Male) = 0.1939551
1861/(7734+1861)

0.19395518499218342

In [12]:
# P(Churn==1 | Female)
1126/(5208+1126)

0.1777707609725292

In [13]:
n1 = male_churn.size
n2 = female_churn.size

test_statistic = (p1 - p2) / np.sqrt((p1 * (1 - p1) / n1) + (p2 * (1 - p2) / n2))
test_statistic

2.579364689811137

In [18]:
# Step 5: Determine the critical value or p-value
critical_value = norm.isf(alpha/2)  # For two-tailed test
critical_value

1.9599639845400545

In [19]:
p_value = 2 * (norm.sf(np.abs(test_statistic)))  # For two-tailed test
p_value

0.00989822278189378

In [20]:
# Step 6: Make a decision
if np.abs(test_statistic) > critical_value or p_value < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")

# Step 7: Draw a conclusion
if np.abs(test_statistic) > critical_value or p_value < alpha:
    print("There is evidence to suggest that gender has an effect on churn")
else:
    print("There is insufficient evidence to suggest a difference in churn rates based on gender")

Reject the null hypothesis
There is evidence to suggest that gender has an effect on churn
